# Importing the libraries

In [ ]:
import numpy as np
import random
import os
import sys
import tensorflow as tf
from tensorflow import keras
import platform
from tensorflow.keras import layers, optimizers, losses, models, Input, Model
from tensorflow.keras.regularizers import l2
import time # Per misurare il tempo di training
from tensorflow.keras.callbacks import EarlyStopping # Per l'early stopping
import matplotlib.pyplot as plt
from tqdm import tqdm # Per mostrare una barra di progresso

# Set seeds for random operations.

In [ ]:
M=4
BETA = 50

# --- 1. Impostazione del Seed Globale all'inizio del tuo script ---
# Questo è il punto chiave per la riproducibilità di TUTTO ciò che segue.
MASTER_RANDOM_SEED = 42
np.random.seed(MASTER_RANDOM_SEED)
random.seed(MASTER_RANDOM_SEED) # Imposta anche il seed per la libreria 'random' di Python se la usi
tf.random.set_seed(MASTER_RANDOM_SEED)
os.environ['PYTHONHASHSEED'] = str(MASTER_RANDOM_SEED) # Per operazioni basate su hash (es. ordine dei dizionari)
os.environ['TF_DETERMINISTIC_OPS'] = '1' # Forza operazioni deterministiche in TensorFlow 2.x

# Print the HW Specs.

In [ ]:
print("--- Dettagli dell'Architettura Hardware della Sessione Colab ---\n")

# --- 1. Dettagli CPU ---
print("--- Dettagli CPU ---")
!lscpu
print("\n")

# --- 2. Dettagli RAM (Memoria) ---
print("--- Dettagli RAM (Memoria) ---")
!cat /proc/meminfo | grep MemTotal
print("\n")

# --- 3. Dettagli Spazio su Disco ---
print("--- Dettagli Spazio su Disco ---")
!df -h /
print("\n")

# --- 4. Dettagli Acceleratore Hardware (GPU/TPU) ---
print("--- Dettagli Acceleratore Hardware (GPU/TPU) ---")
try:
    tpu_address = os.environ.get('COLAB_TPU_ADDR')
    if tpu_address:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=tpu_address)
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        print(f"Tipo Acceleratore: TPU (indirizzo: {tpu_address})")
        print("Dispositivi TPU disponibili:")
        for device in tf.config.list_logical_devices('TPU'):
            print(f"  - {device.name}")
    else:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            print(f"Tipo Acceleratore: GPU")
            for gpu in gpus:
                print(f"  - Dispositivo GPU rilevato: {gpu.name}")
            print("\nDettagli GPU specifici (da `!nvidia-smi`):")
            !nvidia-smi
        else:
            print("Tipo Acceleratore: Nessuna GPU o TPU rilevata (in uso CPU)")

except Exception as e:
    print(f"Si è verificato un errore durante la rilevazione dell'acceleratore: {e}")
    print("Tentativo di rilevare i dispositivi TensorFlow standard:")
    devices = tf.config.list_logical_devices()
    if devices:
        for device in devices:
            print(f"  - Dispositivo rilevato: {device.name}, Tipo: {device.device_type}")
    else:
        print("Nessun dispositivo TensorFlow rilevato.")

print("\n--- Analisi Dettagli Hardware Completata ---")

--- Dettagli dell'Architettura Hardware della Sessione Colab ---

--- Dettagli CPU ---
Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.00GHz
    CPU family:           6
    Model:                85
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             3
    BogoMIPS:             4000.38
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht sysc
                          all nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xt
                          opology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq
                           ssse3 fma cx16 pcid sse4_1 sse4

# Connect To Gdrive to store the datasets created.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Define the paths

In [ ]:
# Percorsi dataset
paths = {

    #"0-20": {
    #    "train": "/content/drive/MyDrive/GitHub/Rayleigh/dataset/UNIFORM_SNR_SAMPLES_training_0-20_SNR_50_BETA.npz",
    #    "val": "/content/drive/MyDrive/GitHub/Rayleigh/dataset/UNIFORM_SNR_SAMPLES_validation_0-20_SNR_50_BETA.npz",
    #},
    "11-15": {
         "train": "/content/drive/MyDrive/GitHub/Rayleigh/dataset/training_11-15_SNR_50_BETA.npz",
         "val": "/content/drive/MyDrive/GitHub/Rayleigh/dataset/validation_11-15_SNR_50_BETA.npz",
    }
}

# Directory salvataggio modelli
save_dir = "/content/drive/MyDrive/GitHub/Rayleigh/trained_models/benchmarks"
os.makedirs(save_dir, exist_ok=True)

# Load the datasets.

In [ ]:
def load_dataset(filepath):
    """
    Carica i dati X e y da un file .npz.
    Si aspetta che il file contenga 'X_train' o 'X_val' e 'y_train' o 'y_val'.
    """
    data = np.load(filepath)

    # Controlla se le chiavi per il training sono presenti
    if 'X_train' in data and 'y_train' in data:
        print(f"  Caricato Training data da: {filepath}")
        return data['X_train'], data['y_train']
    # Altrimenti, controlla se le chiavi per la validation sono presenti
    elif 'X_val' in data and 'y_val' in data:
        print(f"  Caricato Validation data da: {filepath}")
        return data['X_val'], data['y_val']
    else:
        # Se nessuna delle combinazioni attese è trovata, solleva un errore
        raise ValueError(f"Il file {filepath} non contiene i dati X e y attesi (né 'X_train'/'y_train' né 'X_val'/'y_val'). "
                         f"Chiavi trovate: {list(data.keys())}")

# Model version definitions.

In [ ]:
# --- Funzioni per definire i modelli ---

def get_two_conv1d_global_avg_pool_attention_model(input_shape, model_name='TwoConv1D_GlobalAvgPool_Attention'):
    inputs = Input(shape=input_shape)
    x = layers.Reshape((input_shape[0], 1), name="reshape_for_conv")(inputs)
    x = layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same', name="conv1d_layer_1")(x)
    conv_output_2 = layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', name="conv1d_layer_2")(x)

    attention_weights = layers.Conv1D(filters=1, kernel_size=1, activation='sigmoid', name="attention_weights")(conv_output_2)
    attended_features = layers.Multiply(name="multiply_attention")([conv_output_2, attention_weights])

    x = layers.GlobalAveragePooling1D(name="global_average_pooling_layer")(attended_features)
    x = layers.Dropout(0.3, name="dropout_layer")(x)
    x = layers.Dense(128, activation='relu', name="fc1")(x)
    outputs = layers.Dense(2, activation='softmax', name="output_softmax")(x)
    model = Model(inputs, outputs, name=model_name)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def get_two_conv1d_global_avg_pool_attention_light_model(input_shape, model_name='TwoConv1D_GlobalAvgPool_Attention_Light'):
    inputs = Input(shape=input_shape)
    x = layers.Reshape((input_shape[0], 1), name="reshape_for_conv")(inputs)
    x = layers.Conv1D(filters=24, kernel_size=3, activation='relu', padding='same', name="conv1d_layer_1")(x)
    conv_output_2 = layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same', name="conv1d_layer_2")(x)

    attention_weights = layers.Conv1D(filters=1, kernel_size=1, activation='sigmoid', name="attention_weights")(conv_output_2)
    attended_features = layers.Multiply(name="multiply_attention")([conv_output_2, attention_weights])

    x = layers.GlobalAveragePooling1D(name="global_average_pooling_layer")(attended_features)
    x = layers.Dropout(0.3, name="dropout_layer")(x)
    x = layers.Dense(64, activation='relu', name="fc1")(x)
    outputs = layers.Dense(2, activation='softmax', name="output_softmax")(x)
    model = Model(inputs, outputs, name=model_name)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def get_two_conv1d_global_avg_pool_attention_heavy_model(input_shape, model_name='TwoConv1D_GlobalAvgPool_Attention_Heavy'):
    inputs = Input(shape=input_shape)
    x = layers.Reshape((input_shape[0], 1), name="reshape_for_conv")(inputs)
    x = layers.Conv1D(filters=48, kernel_size=3, activation='relu', padding='same', name="conv1d_layer_1")(x)
    conv_output_2 = layers.Conv1D(filters=96, kernel_size=3, activation='relu', padding='same', name="conv1d_layer_2")(x)

    attention_weights = layers.Conv1D(filters=96, kernel_size=1, activation='sigmoid', name="attention_weights")(conv_output_2)
    attended_features = layers.Multiply(name="multiply_attention")([conv_output_2, attention_weights])

    x = layers.GlobalAveragePooling1D(name="global_average_pooling_layer")(attended_features)
    x = layers.Dropout(0.3, name="dropout_layer")(x)
    x = layers.Dense(128, activation='relu', name="fc1")(x)
    outputs = layers.Dense(2, activation='softmax', name="output_softmax")(x)
    model = Model(inputs, outputs, name=model_name)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def get_two_conv1d_global_avg_pool_attention_TD_model(input_shape, model_name='TwoConv1D_GlobalAvgPool_Attention_TD'):
    inputs = Input(shape=input_shape)

    # Reshape input to (timesteps, 1) if it's 1D
    x = layers.Reshape((input_shape[0], 1), name="reshape_for_conv")(inputs)  # (batch, time_steps, 1)

    # Conv layers
    x = layers.Conv1D(filters=48, kernel_size=3, activation='relu', padding='same', name="conv1d_layer_1")(x)
    conv_output_2 = layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', name="conv1d_layer_2")(x)

    # Attention
    attention_weights = layers.Conv1D(filters=64, kernel_size=1, activation='sigmoid', name="attention_weights")(conv_output_2)
    attended_features = layers.Multiply(name="multiply_attention")([conv_output_2, attention_weights])  # shape: (batch, time_steps, 64)

    # TimeDistributed Dense
    x = layers.TimeDistributed(layers.Dense(32, activation='relu'), name="td_dense_1")(attended_features)  # still (batch, time_steps, 32)

    # GlobalAveragePooling1D to collapse time dimension
    x = layers.GlobalAveragePooling1D(name="global_average_pooling_layer")(x)  # (batch, 32)

    # FC layers
    x = layers.Dropout(0.3, name="dropout_layer")(x)
    x = layers.Dense(12, activation='relu', name="fc1")(x)
    outputs = layers.Dense(2, activation='softmax', name="output_softmax")(x)

    model = Model(inputs, outputs, name=model_name)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model


# --- Dizionario dei modelli ---

models_dict = {
    "TwoConv1D_GlobalAvgPool_Attention": get_two_conv1d_global_avg_pool_attention_model,
    "TwoConv1D_GlobalAvgPool_Attention_Light": get_two_conv1d_global_avg_pool_attention_light_model,
    #"TwoConv1D_GlobalAvgPool_Attention_Heavy": get_two_conv1d_global_avg_pool_attention_heavy_model,
    "TwoConv1D_GlobalAvgPool_Attention_TD": get_two_conv1d_global_avg_pool_attention_TD_model,

}

# Training per 30 Epoche.

In [ ]:
# Addestramento
for snr_range, datasets in paths.items():
    x_train, y_train = load_dataset(datasets['train'])
    x_val, y_val = load_dataset(datasets['val'])

    for model_name, builder in models_dict.items():
        print(f"\nAddestramento modello: {model_name} | SNR range: {snr_range}")
        model = builder(input_shape=x_train.shape[1:])

        callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        start_time = time.time() # Registra il tempo di inizio training

        model.fit(
             x_train, y_train,
             validation_data=(x_val, y_val),
             epochs=30,
             batch_size=25,
             callbacks=[callback],
             verbose=1
        )

        end_time = time.time() # Registra il tempo di fine training
        total_training_time = end_time - start_time

        print(f"Training completato.")
        print(f"Tempo totale di training: {total_training_time:.2f} secondi.") # Stampa il tempo totale

        save_path = os.path.join(save_dir, f"{model_name.replace(' ', '_')}-ORIGINALv2-50-BETA_rayleigh_RANDOM_snr_{snr_range}.h5")
        model.save(save_path)
        #ber_values, snr_points = calculate_ber(model, beta=BETA, m =M)

        print(f"Salvato modello in: {save_path}")

  Caricato Training data da: /content/drive/MyDrive/GitHub/Rayleigh/dataset/training_11-15_SNR_50_BETA.npz
  Caricato Validation data da: /content/drive/MyDrive/GitHub/Rayleigh/dataset/validation_11-15_SNR_50_BETA.npz

Addestramento modello: TwoConv1D_GlobalAvgPool_Attention | SNR range: 11-15
Epoch 1/30
2400/2400 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.8629 - loss: 0.2635 - val_accuracy: 0.9714 - val_loss: 0.0647
Epoch 2/30
2400/2400 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - accuracy: 0.9715 - loss: 0.0693 - val_accuracy: 0.9786 - val_loss: 0.0504
Epoch 3/30
2400/2400 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - accuracy: 0.9739 - loss: 0.0630 - val_accuracy: 0.9747 - val_loss: 0.0618
Epoch 4/30
2400/2400 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.9755 - loss: 0.0589 - val_accuracy: 0.9769 - val_loss: 0.0526
Epoch 5/30
2400/2400 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - accuracy: 0.9749 - loss: 0.0583 - val_accuracy: 0.9765 - val_loss: 0.0521
Epoch 6/30
2400/2400 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/

Training completato.
Tempo totale di training: 573.78 secondi.
Salvato modello in: /content/drive/MyDrive/GitHub/Rayleigh/trained_models/benchmarks/TwoConv1D_GlobalAvgPool_Attention-ORIGINALv2-50-BETA_rayleigh_RANDOM_snr_11-15.h5

Addestramento modello: TwoConv1D_GlobalAvgPool_Attention_Light | SNR range: 11-15
Epoch 1/30
2400/2400 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.8519 - loss: 0.3003 - val_accuracy: 0.9728 - val_loss: 0.0659
Epoch 2/30
2400/2400 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.9685 - loss: 0.0743 - val_accuracy: 0.9749 - val_loss: 0.0590
Epoch 3/30
2400/2400 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.9704 - loss: 0.0708 - val_accuracy: 0.9764 - val_loss: 0.0543
Epoch 4/30
2400/2400 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - accuracy: 0.9710 - loss: 0.0684 - val_accuracy: 0.9764 - val_loss: 0.0558
Epoch 5/30
2400/2400 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.9732 - loss: 0.0637 - val_accuracy: 0.9765 - val_loss: 0.0541
Epoch 6/30
2400/2400 ━━━━━━━━━━━

Training completato.
Tempo totale di training: 598.19 secondi.
Salvato modello in: /content/drive/MyDrive/GitHub/Rayleigh/trained_models/benchmarks/TwoConv1D_GlobalAvgPool_Attention_Light-ORIGINALv2-50-BETA_rayleigh_RANDOM_snr_11-15.h5

Addestramento modello: TwoConv1D_GlobalAvgPool_Attention_TD | SNR range: 11-15
Epoch 1/30
2400/2400 ━━━━━━━━━━━━━━━━━━━━ 42s 15ms/step - accuracy: 0.8844 - loss: 0.2415 - val_accuracy: 0.9780 - val_loss: 0.0530
Epoch 2/30
2400/2400 ━━━━━━━━━━━━━━━━━━━━ 40s 15ms/step - accuracy: 0.9718 - loss: 0.0659 - val_accuracy: 0.9783 - val_loss: 0.0497
Epoch 3/30
2400/2400 ━━━━━━━━━━━━━━━━━━━━ 42s 15ms/step - accuracy: 0.9744 - loss: 0.0599 - val_accuracy: 0.9783 - val_loss: 0.0503
Epoch 4/30
2400/2400 ━━━━━━━━━━━━━━━━━━━━ 42s 16ms/step - accuracy: 0.9754 - loss: 0.0576 - val_accuracy: 0.9805 - val_loss: 0.0449
Epoch 5/30
2400/2400 ━━━━━━━━━━━━━━━━━━━━ 40s 15ms/step - accuracy: 0.9769 - loss: 0.0543 - val_accuracy: 0.9802 - val_loss: 0.0444
Epoch 6/30
2400/2400 ━━━

Training completato.
Tempo totale di training: 770.32 secondi.
Salvato modello in: /content/drive/MyDrive/GitHub/Rayleigh/trained_models/benchmarks/TwoConv1D_GlobalAvgPool_Attention_TD-ORIGINALv2-50-BETA_rayleigh_RANDOM_snr_11-15.h5
